In [25]:
from tensorflow.keras import layers, models
from tensorflow.keras.datasets import cifar10
import numpy as np

In [26]:
# Chargement des données CIFAR-10
(train_images, train_labels), (test_images, test_labels) = cifar10.load_data()

In [27]:
# Prétraitement des images (normalisation)
train_images = (train_images.astype('float32') - 127.5) / 127.5
test_images = (test_images.astype('float32') - 127.5) / 127.5

In [28]:
# réseau de neurones convolutionnel (CNN)
cnn_input = layers.Input(shape=(32, 32, 3))
x = layers.Conv2D(32  , (3, 3), activation='relu')(cnn_input)
x = layers.MaxPooling2D((2, 2))(x)
x = layers.Conv2D(256, (3, 3), activation='relu')(x)
x = layers.MaxPooling2D((2, 2))(x)
x = layers.Conv2D(128, (3, 3), activation='relu')(x)
x = layers.Flatten()(x)
cnn_output = layers.Dense(256 , activation='relu')(x)

In [29]:
# Définition des deux branches du réseau siamois
siam_input_a = layers.Input(shape=(32, 32, 3))
siam_input_b = layers.Input(shape=(32, 32, 3))

In [30]:
# CNN sur le reseau siamois
siam_output_a = layers.Dense(64, activation='relu')(layers.Flatten()(layers.Conv2D(32, (3, 3), activation='relu')(siam_input_a)))
siam_output_b = layers.Dense(64, activation='relu')(layers.Flatten()(layers.Conv2D(32, (3, 3), activation='relu')(siam_input_b)))

In [31]:
# Concaténation des sorties des deux trucs
combined_features = layers.Concatenate()([siam_output_a, siam_output_b])

In [32]:
# Fusion des features
merged_features = layers.Concatenate()([cnn_output, combined_features])

In [33]:
# Couche pleinement connecté pour la classification
x = layers.Dense(64, activation='relu')(merged_features)
output = layers.Dense(10, activation='sigmoid')(x)


In [34]:
# Création du modèle
model = models.Model(inputs=[cnn_input, siam_input_a, siam_input_b], outputs=output)

In [35]:
# Compilation
model.compile(optimizer='nadam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [36]:
# Entraînement
model.fit([train_images, train_images, train_images], train_labels, epochs=15, batch_size=128 , validation_data=([test_images, test_images, test_images], test_labels))

Epoch 1/15
391/391 [==============================] - 88s 190ms/step - loss: 1.3290 - accuracy: 0.5315 - val_loss: 1.0458 - val_accuracy: 0.6323
Epoch 2/15
391/391 [==============================] - 67s 170ms/step - loss: 0.8645 - accuracy: 0.7011 - val_loss: 0.8300 - val_accuracy: 0.7134
Epoch 3/15
391/391 [==============================] - 65s 166ms/step - loss: 0.6447 - accuracy: 0.7763 - val_loss: 0.7934 - val_accuracy: 0.7311
Epoch 4/15
391/391 [==============================] - 65s 166ms/step - loss: 0.4863 - accuracy: 0.8310 - val_loss: 0.7618 - val_accuracy: 0.7465
Epoch 5/15
391/391 [==============================] - 66s 169ms/step - loss: 0.3481 - accuracy: 0.8787 - val_loss: 0.8557 - val_accuracy: 0.7453
Epoch 6/15
391/391 [==============================] - 65s 166ms/step - loss: 0.2389 - accuracy: 0.9170 - val_loss: 0.8603 - val_accuracy: 0.7501
Epoch 7/15
391/391 [==============================] - 66s 169ms/step - loss: 0.1608 - accuracy: 0.9442 - val_loss: 1.0363 - val_ac

KeyboardInterrupt: 

In [ ]:
'''from tensorflow.keras.callbacks import ModelCheckpoint

# Définition du chemin de sauvegarde du modèle
checkpoint_path = "best_model.h5"

# Définition du callback ModelCheckpoint pour sauvegarder le meilleur modèle
checkpoint = ModelCheckpoint(filepath=checkpoint_path,
                             monitor='val_accuracy',  # Surveille la précision de validation
                             save_best_only=True,     # Ne sauvegarde que le meilleur modèle
                             verbose=1)               # Affiche des informations lors de la sauvegarde

# Entraînement du modèle avec le callback ModelCheckpoint
history = model.fit([train_images, train_images, train_images], train_labels,
                    epochs=15 , batch_size=128,
                    validation_data=([test_images, test_images, test_images], test_labels),
                    callbacks=[checkpoint])

# Affichage des résultats d'entraînement
print(history.history)

# Chargement du meilleur modèle sauvegardé
best_model = models.load_model(checkpoint_path)

# Évaluation du meilleur modèle sur les données de test
loss, accuracy = best_model.evaluate([test_images, test_images, test_images], test_labels)
print("Meilleur modèle - perte:", loss)
print("Meilleur modèle - précision:", accuracy)'''


In [ ]:
'''from tensorflow.keras.callbacks import ModelCheckpoint

# Définir un rappel pour sauvegarder le meilleur modèle
checkpoint_callback = ModelCheckpoint(filepath='best_model.h5',
                                      monitor='val_accuracy',
                                      mode='max',
                                      save_best_only=True)

# Entraîner le modèle en utilisant le rappel
model.fit(train_images,train_images,train_images,
          validation_data=(test_images,test_labels),
          epochs=10,
          callbacks=[checkpoint_callback])'''


In [ ]:
#best_model = models.load_model('best_model.h5')